<a href="https://colab.research.google.com/github/ChirayuSalgarkar/SafeDiffuseTraj/blob/main/diffmod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 This was figured out as of 4:00:00 AM EST 09-DEC.
 Requirements - Colab Pro for compute power. Was unable to use RIT GPU due to resource allocation issues.
Some notes: This is a modification of the CleanDiffuser paradigm (https://github.com/CleanDiffuserTeam/CleanDiffuser). However, their packages require several extraneous installations. If trying to reproduce, include MuJoco 210 (not 200). If implementing on RC, note the following:

1.   RC does not allow SUDO access, which hinders installation ability, as there exist conflicts between requirements and on-board packages.
2.   Therefore, to install, one must create a fake-root in Apptainer, RIT's supported container system environment. This requires some energy.
3.   RC availability is not guaranteed - many are using the systems GPU, and it is unlikely to have significant available allocation for more than an hour.



In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


In [1]:
!conda install pytorch==2.2.2 torchvision==0.17.2 pytorch-cuda=12.1 -c pytorch -c nvidia

Channels:
 - pytorch
 - nvidia
 - conda-forge
Platform: linux-64
Solving environment: - \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pytorch-cuda=12.1
    - pytorch==2.2.2
    - torchvision==0.17.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    blas-2.116                 |              mkl          13 KB  conda-forge
    blas-devel-3.9.0           |   16_linux64_mkl          12 KB  conda-forge
    ca-certificates-2024.8.30  |       hbcca054_0         155 KB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    cuda

In [2]:
!conda install -c conda-forge libstdcxx-ng=12

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - libstdcxx-ng=12


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    gmpy2-2.1.5                |  py310hc7909c9_1         201 KB  conda-forge
    libgcc-12.4.0              |       h77fa898_1         764 KB  conda-forge
    libgcc-ng-12.4.0           |       h69a702a_1          51 KB  conda-forge
    libgfortran-12.4.0         |       h69a702a_1          51 KB  conda-forge
    libgfortran-ng-12.4.0      |       h69a702a_1          51 KB  conda-forge
    libgfortran5-12.4.0        |       h583ef98_1        

In [3]:
!rm -rf ~/.mujoco
!mkdir -p ~/.mujoco
!wget https://github.com/deepmind/mujoco/releases/download/2.1.0/mujoco210-linux-x86_64.tar.gz
!tar xvzf mujoco210-linux-x86_64.tar.gz -C ~/.mujoco
!rm mujoco210-linux-x86_64.tar.gz
!wget -P ~/.mujoco https://www.roboti.us/file/mjkey.txt

--2024-12-09 09:13:34--  https://github.com/deepmind/mujoco/releases/download/2.1.0/mujoco210-linux-x86_64.tar.gz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/google-deepmind/mujoco/releases/download/2.1.0/mujoco210-linux-x86_64.tar.gz [following]
--2024-12-09 09:13:34--  https://github.com/google-deepmind/mujoco/releases/download/2.1.0/mujoco210-linux-x86_64.tar.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/400501136/1f51148e-4e64-4a12-a400-d6f1e21be444?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241209%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20241209T091334Z&X-Amz-Expires=300&X-Amz-Signature=61e91ad2306ef089c93e80f35a9c8edd7ce7d3ea0076210a54d7450f

In [4]:
!conda create -n cleandiffuser python==3.9

Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/cleandiffuser

  added / updated specs:
    - python==3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ld_impl_linux-64-2.43      |       h712a8e2_2         654 KB  conda-forge
    libffi-3.3                 |       h58526e2_2          51 KB  conda-forge
    liblzma-5.6.3              |       hb9d3cd8_1         109 KB  conda-forge
    liblzma-devel-5.6.3        |       hb9d3cd8_1         368 KB  conda-forge
    libsqlite-3.46.0           |       hde9e2c9_0         845 KB  conda-forge
    libstdcxx-ng-14.2.0        |       h4852527_1          53 KB  conda-for

In [5]:
!apt install libosmesa6-dev libgl1-mesa-glx libglfw3 libglew-dev patchelf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdrm-dev libgl-dev libglu1-mesa libglu1-mesa-dev libglx-dev libosmesa6 libpciaccess-dev
  mesa-common-dev
Suggested packages:
  libgles1 libvulkan1
The following NEW packages will be installed:
  libdrm-dev libgl-dev libgl1-mesa-glx libglew-dev libglfw3 libglu1-mesa libglu1-mesa-dev
  libglx-dev libosmesa6 libosmesa6-dev libpciaccess-dev mesa-common-dev patchelf
0 upgraded, 13 newly installed, 0 to remove and 49 not upgraded.
Need to get 6,592 kB of archives.
After this operation, 23.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpciaccess-dev amd64 0.16-3 [21.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libdrm-dev amd64 2.4.113-2~ubuntu0.22.04.1 [292 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:4 http

In [6]:
!git clone https://github.com/CleanDiffuserTeam/CleanDiffuser

Cloning into 'CleanDiffuser'...
remote: Enumerating objects: 1936, done.
remote: Counting objects: 100% (415/415), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 1936 (delta 262), reused 292 (delta 180), pack-reused 1521 (from 1)
Receiving objects: 100% (1936/1936), 19.87 MiB | 38.83 MiB/s, done.
Resolving deltas: 100% (1007/1007), done.


In [7]:
!apt install cmake build-essential gcc g++

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
gcc is already the newest version (4:11.2.0-1ubuntu1).
gcc set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [8]:
!cd CleanDiffuser; conda run -n cleandiffuser pip install -e .

Obtaining file:///content/CleanDiffuser
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.2/626.2 kB 15.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached packaging-24.2-py3-none-any.whl.meta

In [9]:
!git clone https://github.com/Farama-Foundation/D4RL.git; cd D4RL; conda run -n cleandiffuser pip install -e .
!git clone https://github.com/ARISE-Initiative/robomimic.git; cd robomimic; conda run -n cleandiffuser pip install -e .
!git clone https://github.com/ARISE-Initiative/robosuite.git; cd robosuite; conda run -n cleandiffuser pip install -e .

Cloning into 'D4RL'...
remote: Enumerating objects: 1384, done.
remote: Counting objects: 100% (637/637), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 1384 (delta 394), reused 351 (delta 350), pack-reused 747 (from 1)
Receiving objects: 100% (1384/1384), 28.55 MiB | 44.50 MiB/s, done.
Resolving deltas: 100% (718/718), done.
Obtaining file:///content/D4RL
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning https://github.com/aravindr93/mjrl (to revision master) to /tmp/pip-install-afapbizu/mjrl_73595ea48cff4314bfa2592167a2db3e
  Resolved https://github.com/aravindr93/mjrl to commit 3871d93763d3b49c4741e6daeaebbc605fe140dc
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 134.0 MB/s eta 0:00:00
  Created wheel for mjrl

Please run this in terminal.

In [10]:
!LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/root/.mujoco/mujoco210/bin conda run --live-stream -n cleandiffuser --cwd CleanDiffuser python pipelines/diffuser_d4rl_antmaze.py mode=train

These new versions include large bug fixes, new versions of Python, and are where all new development will continue. Please upgrade these libraries as soon as you're able to do so.
If you'd like to read more about the story behind this switch, please check out ]8;;https://farama.org/Announcing-Minari\this blog post]8;;\.
/usr/local/envs/cleandiffuser/lib/python3.9/site-packages/Cython/Distutils/old_build_ext.py:14: DeprecationWarning: dep_util is Deprecated. Use functions from setuptools instead.
  from distutils.dep_util import newer, newer_group
Compiling /usr/local/envs/cleandiffuser/lib/python3.9/site-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/envs/cleandiffuser/lib/python3.9/site-packages/mujoco_py/cymj.pyx
No module named 'flow'
INFO:absl:MUJOCO_GL is not set, so an OpenGL backend will be chosen automatically.
/usr/local/envs/cleandiffuser/lib/python3.9/site-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment vari